# Openclassrooms PJ4 : transats dataset : modelisation notebook

In [1]:
%matplotlib inline

import os
import zipfile
import urllib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import qgrid

import glob

from pandas.plotting import scatter_matrix

DATA_PATH = os.path.join("datasets", "transats")
DATA_PATH = os.path.join(DATA_PATH, "out")

DATA_PATH_FILE_INPUT = os.path.join(DATA_PATH, "transats_metadata_transformed.csv")

plt.rcParams["figure.figsize"] = [16,9] # Taille par défaut des figures de matplotlib

import seaborn as sns
sns.set()

#import common_functions

In [2]:
def qgrid_show(df):
    display(qgrid.show_grid(df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 170}))

In [3]:
def print_column_information(df, column_name):
    column_type = df.dtypes[column_name]
    print(f'Column {column_name}, type {column_type}\n')
    print('--------------------------')

    print(df[[column_name]].groupby(column_name).size().sort_values(ascending=False))
    print(df[column_name].unique())    
    print('\n')


In [4]:
def display_percent_complete(df):
    not_na = 100 - (df.isnull().sum() * 100 / len(df))
    not_na_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_complete': not_na}).sort_values(by='percent_complete', ascending=False)
    display(not_na_df)

In [5]:
def identify_features(df, all_features):
    quantitative_features = []
    qualitative_features = []
    features_todrop = []

    for feature_name in all_features:
        if (df[feature_name].dtype == 'object'):
            qualitative_features.append(feature_name)

        else:
            quantitative_features.append(feature_name)

    print(f'Quantitative features : {quantitative_features} \n')
    print(f'Qualitative features : {qualitative_features} \n')  
    
    return quantitative_features, qualitative_features

# Data load

In [6]:
# hhmm timed features formatted
feats_hhmm = ['CRS_DEP_TIME',  'CRS_ARR_TIME']

df = pd.read_csv(DATA_PATH_FILE_INPUT, sep=',', header=0, encoding='utf-8', low_memory=False, parse_dates=feats_hhmm)

In [7]:
df.shape

(5547828, 14)

In [8]:
display_percent_complete(df)

,column_name,percent_complete
ORIGIN,ORIGIN,100.0
CRS_DEP_TIME,CRS_DEP_TIME,100.0
MONTH,MONTH,100.0
DAY_OF_MONTH,DAY_OF_MONTH,100.0
DAY_OF_WEEK,DAY_OF_WEEK,100.0
UNIQUE_CARRIER,UNIQUE_CARRIER,100.0
DEST,DEST,100.0
CRS_ARR_TIME,CRS_ARR_TIME,100.0
DISTANCE,DISTANCE,100.0
CRS_ELAPSED_TIME,CRS_ELAPSED_TIME,100.0


In [9]:
for column_name in df.columns:
    print_column_information(df, column_name)

Column ORIGIN, type object

--------------------------
ORIGIN
ATL    392037
ORD    240967
DEN    222012
LAX    210219
DFW    194370
        ...  
MMH        91
GST        83
DLG        83
PGD        78
AKN        63
Length: 308, dtype: int64
['DFW' 'DTW' 'SEA' 'JFK' 'SJC' 'ORD' 'PHX' 'STL' 'LAX' 'MCO' 'DEN' 'MIA'
 'KOA' 'IAH' 'AUS' 'LAS' 'SLC' 'TUS' 'STT' 'BOS' 'FLL' 'SFO' 'OGG' 'TPA'
 'SNA' 'OKC' 'HNL' 'PHL' 'LGA' 'RDU' 'DCA' 'RIC' 'ATL' 'LBB' 'CLT' 'ELP'
 'SAN' 'BNA' 'JAC' 'SMF' 'EWR' 'IAD' 'LIH' 'SJU' 'ABQ' 'ORF' 'JAX' 'MSY'
 'SAT' 'MCI' 'GUC' 'IND' 'PDX' 'BWI' 'MSP' 'MKE' 'TUL' 'ONT' 'RSW' 'RNO'
 'DSM' 'MFE' 'PSP' 'OMA' 'EGE' 'PBI' 'SDF' 'PIT' 'FAT' 'DAY' 'STX' 'COS'
 'CMH' 'MTJ' 'HDN' 'BDL' 'MEM' 'CLE' 'HOU' 'BOI' 'OAK' 'GEG' 'ANC' 'BUF'
 'SYR' 'ALB' 'PVD' 'ROC' 'ILM' 'ICT' 'PWM' 'GSO' 'CHS' 'MDT' 'BHM' 'ADQ'
 'BET' 'BRW' 'SCC' 'FAI' 'JNU' 'KTN' 'YAK' 'CDV' 'SIT' 'PSG' 'WRG' 'OME'
 'OTZ' 'BUR' 'BLI' 'ADK' 'SWF' 'LGB' 'PSE' 'BQN' 'HPN' 'SAV' 'SRQ' 'BTV'
 'ORH' 'DAB' 'CVG' 'BIS' 'AV

# Identification of features

In [10]:
# Below are feature from dataset that we decided to keep: 
all_features = ['ORIGIN','CRS_DEP_TIME','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','UNIQUE_CARRIER','DEST','CRS_ARR_TIME','DISTANCE','CRS_ELAPSED_TIME','ARR_DELAY','DEP_DELAY', 'TAXI_OUT', 'TAIL_NUM']

model1_features = ['ORIGIN','CRS_DEP_TIME','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','UNIQUE_CARRIER','DEST','CRS_ARR_TIME','DISTANCE','CRS_ELAPSED_TIME']
model1_label = 'ARR_DELAY'

quantitative_features = []
qualitative_features = []
features_todrop = []

for feature_name in all_features:
    if (df[feature_name].dtype == 'object'):
        qualitative_features.append(feature_name)
        
    else:
        quantitative_features.append(feature_name)

print(f'Quantitative features : {quantitative_features} \n')
print(f'Qualitative features : {qualitative_features} \n')        
        

#Commented out : no drop of features
#for df_column in df.columns:
#    if df_column not in all_features:
#        features_todrop.append(df_column)
#        
#print(f'Features to drop : {features_todrop} \n')


Quantitative features : ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DISTANCE', 'CRS_ELAPSED_TIME', 'ARR_DELAY', 'DEP_DELAY', 'TAXI_OUT'] 

Qualitative features : ['ORIGIN', 'CRS_DEP_TIME', 'UNIQUE_CARRIER', 'DEST', 'CRS_ARR_TIME', 'TAIL_NUM'] 



# Split train set, test set

In [185]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [186]:
df_train = df_train.sample(1000).copy(deep=True)

In [187]:
df_train

,ORIGIN,CRS_DEP_TIME,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,DEST,CRS_ARR_TIME,DISTANCE,CRS_ELAPSED_TIME,ARR_DELAY,DEP_DELAY,TAXI_OUT,TAIL_NUM
3943445,LAX,1606,9,8,4,UA,EWR,0021,2454.0,315.0,-3.0,11.0,18.0,N17105
2792288,CLE,2230,7,11,1,WN,BWI,2340,314.0,70.0,-5.0,-3.0,10.0,N764SW
4807829,SAN,1830,11,15,2,WN,PHX,2040,304.0,70.0,-2.0,3.0,11.0,N620SW
1713378,ATL,1350,3,17,4,DL,DEN,1512,1199.0,202.0,15.0,3.0,23.0,N935DN
4964824,SFO,1900,11,9,3,UA,OGG,2236,2338.0,336.0,-16.0,0.0,13.0,N87512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795052,CLE,1528,9,28,3,AA,CLT,1708,430.0,100.0,6.0,-5.0,13.0,N840AW
5062795,DFW,1220,11,11,5,AA,LAX,1356,1235.0,216.0,5.0,18.0,24.0,N858AA
1711453,MCO,1725,3,17,4,DL,ATL,1907,404.0,102.0,13.0,19.0,25.0,N6708D
5401696,LGA,0840,12,21,3,UA,IAH,1158,1416.0,258.0,-48.0,-1.0,13.0,N78509


# Features encoding

In [188]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn import decomposition
from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer

import statistics


'''
Cette fonction fait un 1 hot encoding des features qui sont des catégories
Elle fonctionne pour les 2 cas de figure suivant :
- Les valeurs possibles de la colonne sont une chaîne de caractère (ex : cat1)
- Les valeurs possibles de la colonne sont des chaînes de caractère avec des séparateurs (ex:  cat1|cat2|cat3)
'''
    
def add_categorical_features_1hot(df, categorical_features_totransform):
    #df.drop(labels=categorical_features_totransform, axis=1, inplace=True)
    
    for feature_totransform in categorical_features_totransform:
        print(f'Adding 1hot Feature : {feature_totransform}')
        
        df_transformed = df[feature_totransform].str.get_dummies().add_prefix(feature_totransform +'_')   
        df.drop(labels=feature_totransform, axis=1, inplace=True)
        
        df = pd.concat([df, df_transformed], axis=1)
        
    return(df)


class HHMM_to_Minutes(BaseEstimator, TransformerMixin):
    def __init__(self, features_toconvert = ['CRS_DEP_TIME', 'CRS_ARR_TIME']):
        self.features_toconvert = features_toconvert
        return None
    
    def fit(self, df):      
        return self
    
    def transform(self, df):       
        for feature_toconvert in self.features_toconvert:
            df_concat = pd.concat([df[feature_toconvert].str.slice(start=0,stop=2, step=1),df[feature_toconvert].str.slice(start=2,stop=4, step=1)], axis=1).astype(int)
            df[feature_toconvert] = (df_concat.iloc[:, [0]] * 60 + df_concat.iloc[:, [1]])[feature_toconvert]
        
        return(df)

        
class CategoricalFeatures1HotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features_totransform=['ORIGIN', 'UNIQUE_CARRIER', 'DEST']):
        self.categorical_features_totransform = categorical_features_totransform
    
    def fit(self, df, labels=None):      
        return self
    
    def transform(self, df):       
        return(add_categorical_features_1hot(df, self.categorical_features_totransform))

class FeaturesSelector(BaseEstimator, TransformerMixin):
    def __init__(self, features_toselect = None):  # If None : every column is kept, nothing is done
        self.features_toselect = features_toselect
    
    def fit(self, df, labels=None):      
        return self
    
    def transform(self, df):       
        if (self.features_toselect != None):
            filter_cols = [col for col in df if (col.startswith(tuple(self.features_toselect)))]
            return(df[filter_cols])    

        else:
            return(df)
    
preparation_pipeline = Pipeline([
    ('data_converter', HHMM_to_Minutes()),
    ('categoricalfeatures_1hotencoder', CategoricalFeatures1HotEncoder()),
    #('standardscaler', preprocessing.StandardScaler()),
])


prediction_pipeline = Pipeline([
    ('features_selector', FeaturesSelector(features_toselect=['ORIGIN','CRS_DEP_TIME','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','UNIQUE_CARRIER','DEST','CRS_ARR_TIME','DISTANCE','CRS_ELAPSED_TIME'])),
    ('standardscaler', ColumnTransformer([
        ('standardscaler_specific', StandardScaler(), ['CRS_DEP_TIME','MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK', 'CRS_ARR_TIME', 'DISTANCE', 'CRS_ELAPSED_TIME'])
    ], remainder='passthrough')),
    #('predictor', To_Complete(predictor_params =  {'n_neighbors':6, 'algorithm':'ball_tree', 'metric':'minkowski'})),
])


'''
ColumnTransformer([
        ('standardscaler_specific', StandardScaler(), ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DISTANCE', 'CRS_ELAPSED_TIME', 'ARR_DELAY', 'DEP_DELAY', 'TAXI_OUT'])
    ], remainder='passthrough')
'''

"\nColumnTransformer([\n        ('standardscaler_specific', StandardScaler(), ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DISTANCE', 'CRS_ELAPSED_TIME', 'ARR_DELAY', 'DEP_DELAY', 'TAXI_OUT'])\n    ], remainder='passthrough')\n"

In [189]:
df_transformed = preparation_pipeline.fit_transform(df_train)

Adding 1hot Feature : ORIGIN
Adding 1hot Feature : UNIQUE_CARRIER
Adding 1hot Feature : DEST


In [190]:
df_transformed.shape

(1000, 310)

In [191]:
type(df_transformed)

pandas.core.frame.DataFrame

In [192]:
df_transformed = prediction_pipeline.fit_transform(df_transformed)

In [193]:
df_transformed.shape

(1000, 306)

In [194]:
df_transformed

array([[ 0.49942336,  0.70019342, -0.92917492, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.81753444,  0.12508589, -0.58693738, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.99371502,  1.27530095, -0.13062066, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.77059726, -1.02512917,  0.0975377 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.03150773,  1.56285471,  0.55385442, ...,  0.        ,
         0.        ,  0.        ],
       [-0.33812638,  0.70019342, -0.01654148, ...,  0.        ,
         0.        ,  0.        ]])

In [195]:
pd.set_option('display.max_columns', 400)

In [196]:
quantitative_features, qualitative_features = identify_features(df, all_features)

Quantitative features : ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DISTANCE', 'CRS_ELAPSED_TIME', 'ARR_DELAY', 'DEP_DELAY', 'TAXI_OUT'] 

Qualitative features : ['ORIGIN', 'CRS_DEP_TIME', 'UNIQUE_CARRIER', 'DEST', 'CRS_ARR_TIME', 'TAIL_NUM'] 

